In [ ]:
import pickle
import pandas as pd
import numpy as np
import math
import random
from tqdm import tqdm
from time import sleep
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
with open('../song_data/dense_song_user_edges.dict', 'rb') as handle:
    songs1 = set(pickle.load(handle).keys())
    
with open('../song_data/dense_song_person_edges.dict', 'rb') as handle:
    songs2 = set(pickle.load(handle).keys())

with open('../song_data/dense_user_song_edges.dict', 'rb') as handle:
    user_song = pickle.load(handle)

In [ ]:
# KG and positive
user_song_train = {}
user_song_test = {}
user_song_tuple_train_pos = []
user_song_tuple_test_pos = []

song_id_set = songs1|songs2

for user in tqdm(user_song):
    pos_songs = user_song[user]
    random.shuffle(pos_songs)
    cut = int(len(pos_songs)*0.8)
    user_song_train[user] = pos_songs[:cut]
    for song in pos_songs[:cut]:
        user_song_tuple_train_pos.append([user, song])
    user_song_test[user] = pos_songs[cut:]
    for song in pos_songs[cut:]:
        user_song_tuple_test_pos.append([user, song])
  

In [ ]:
with open('user_song_train_dense.dict', 'wb') as handle:
    pickle.dump(user_song_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_test_dense.dict', 'wb') as handle:
    pickle.dump(user_song_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_train_pos_dense.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_train_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_test_pos_dense.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_test_pos, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# negative
user_song_tuple_train_neg = []
user_song_tuple_test_neg = []

for user in tqdm(user_song):
    pos_song_set = set(user_song[user])
    neg_song_set = song_id_set - pos_song_set
    neg_songs = list(neg_song_set)
    
    train_neg_len = len(user_song_train[user])*4
    test_neg_len = len(user_song_test[user])*100
    total_neg_len = train_neg_len + test_neg_len
        
    if total_neg_len > len(neg_songs):
        neg_songs_shuffled = [neg_songs[i] for i in np.random.choice(len(neg_songs), len(neg_songs), replace=False)]
        if train_neg_len > len(neg_songs):
            user_song_tuple_train_neg.extend([[user, song] for song in neg_songs_shuffled])
        else:
            user_song_tuple_train_neg.extend([[user, song] for song in neg_songs_shuffled[:train_neg_len]])
            user_song_tuple_test_neg.extend([[user, song] for song in neg_songs_shuffled[train_neg_len:]])
    else:
        neg_songs_shuffled_index = np.random.choice(len(neg_songs), total_neg_len, replace=False)
        i = 0
        while i < train_neg_len:
            user_song_tuple_train_neg.append([user, neg_songs[neg_songs_shuffled_index[i]]])
            i += 1
        while i < train_neg_len + test_neg_len:
            user_song_tuple_test_neg.append([user, neg_songs[neg_songs_shuffled_index[i]]])
            i += 1

In [ ]:
with open('user_song_tuple_train_neg_dense.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_train_neg, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('user_song_tuple_test_neg_dense.txt', 'wb') as handle:
    pickle.dump(user_song_tuple_test_neg, handle, protocol=pickle.HIGHEST_PROTOCOL)